In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw

def loadMesh(filename: str) -> list[float]:
    v = []
    vt = []
    vn = []

    vertices = []

    with open(filename, "r") as file:

        line = file.readline()

        while line:

            words = line.split(" ")
            if words[0]=="v":
                v.append(read_vertex_data(words))
            elif words[0]=="f":
                face = []            
                for word in words[1:]:
                    w = word.split('/')

                    face.append(int(w[0]))
                    vertices.append(v[int(w[0])-1])

                # read_face_data(words, v, vertices)
            
            line = file.readline()
    # print(v[1200],v[1201])
    # print(vertices[-3],vertices[-2])
    return vertices
def read_vertex_data(words: list[str]) -> list[float]:
    #вершины
    return [
        float(words[1]),
        float(words[2]),
        float(words[3])
    ]

    
def odnor(coord):
    new=[]
    for i in range(len(coord)):
        new.append(list(coord[i]))
        new[i].append(1)
    return new

In [2]:
from numpy.linalg import norm
def camera(coord,cam,a,img):
    n=len(coord)
    up=np.array([0,1,0])
    # print(cam,up,a)
    w=(a-cam)/norm((a-cam))
    u=np.cross(cam,up)/norm(np.cross(cam,up))
    v=np.cross(w,u)
    preobr=np.eye(4,4)
    preobr[0,:-1]=u
    preobr[1,:-1]=v
    preobr[2,:-1]=w


    print(preobr)
    new=odnor(coord)
    back=[]
    h=img.height/2
    new=np.dot(new,preobr)
    for i in range(n):
        back.append(tuple(new[i][0:-1]/new[i][-1]))
    return back

img = Image.new("RGB",(2400,2400), (255,255,255))
points=(loadMesh('african_head.obj'))
h=img.height/2
new_c= camera(points,np.array([4,4,4]),np.array([0,1,0]),img)
# new_c= points
draw = ImageDraw.Draw(img)
for i in range(0,int(len(new_c))-2,3):
   
    draw.polygon([(600*(2-new_c[i][0]),600*(2-new_c[i][1])),(600*(2-new_c[i+1][0]),600*(2-new_c[i+1][1])),(600*(2-new_c[i+2][0]),600*(2-new_c[i+2][1]))], outline='black')
    
    
img.show()

[[-0.70710678  0.          0.70710678  0.        ]
 [-0.33129458  0.88345221 -0.33129458  0.        ]
 [-0.62469505 -0.46852129 -0.62469505  0.        ]
 [ 0.          0.          0.          1.        ]]


In [3]:
def z_buffer(coord,fon,img):
    z_buffer = [[float('inf') for _ in range(img.width)] for _ in range(img.height)]
    